This file gets data from the internet and prepares it in a form that is convenient for analysis.

In [7]:
import re
import tarfile
import yaml
import pandas as pd
from pathlib import Path
from urllib.parse import urljoin
import bz2
import requests
import pickle
import time
from datetime import datetime

In [8]:
START_YEAR = 2016
BASE_YEAR_URL = "https://data.everef.net/market-history/{year}/"
OUT_DIR = Path("data")
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [9]:

UA = {"User-Agent": "everef-market-history-downloader (python requests)"}

# matches: market-history-YYYY-MM-DD.csv.bz2
FILE_RE = re.compile(r"market-history-\d{4}-\d{2}-\d{2}\.csv\.bz2$")



In [10]:
def list_files(year_url: str) -> list[str]:
    """Return absolute URLs of all matching .csv.bz2 files listed on the year page."""
    r = requests.get(year_url, headers=UA, timeout=60)
    r.raise_for_status()

    # Simple href extraction (works for typical directory listings)
    hrefs = re.findall(r'href="([^"]+)"', r.text)

    files = []
    for href in hrefs:
        name = href.split("/")[-1]
        if FILE_RE.search(name):
            files.append(urljoin(year_url, href))

    # de-dup + stable order
    return sorted(set(files))

In [11]:
def download(url: str, out_dir: Path) -> Path:
    filename = url.split("/")[-1]
    out_path = out_dir / filename

    if out_path.exists() and out_path.stat().st_size > 0:
        print(f"SKIP  {filename}")
        return out_path

    print(f"GET   {filename}")
    with requests.get(url, headers=UA, stream=True, timeout=120) as r:
        r.raise_for_status()
        tmp_path = out_path.with_suffix(out_path.suffix + ".part")
        with open(tmp_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=1024 * 1024):
                if chunk:
                    f.write(chunk)
        tmp_path.replace(out_path)
    time.sleep(0.5)  # be nice to the server
    return out_path

In [12]:

for year in range(START_YEAR, datetime.now().year + 1):
    year_url = BASE_YEAR_URL.format(year=year)
    urls = list_files(year_url)
    for u in urls:
        try:
            download(u, OUT_DIR)
        except Exception as e:
            print(f"FAIL  {u}  ({e})")
    


GET   market-history-2016-01-01.csv.bz2
GET   market-history-2016-01-02.csv.bz2
GET   market-history-2016-01-03.csv.bz2
GET   market-history-2016-01-04.csv.bz2
GET   market-history-2016-01-05.csv.bz2
GET   market-history-2016-01-06.csv.bz2
GET   market-history-2016-01-07.csv.bz2
GET   market-history-2016-01-08.csv.bz2
GET   market-history-2016-01-09.csv.bz2
GET   market-history-2016-01-10.csv.bz2
GET   market-history-2016-01-11.csv.bz2
GET   market-history-2016-01-12.csv.bz2
GET   market-history-2016-01-13.csv.bz2
GET   market-history-2016-01-14.csv.bz2
GET   market-history-2016-01-15.csv.bz2
GET   market-history-2016-01-16.csv.bz2
GET   market-history-2016-01-17.csv.bz2
GET   market-history-2016-01-18.csv.bz2
GET   market-history-2016-01-19.csv.bz2
GET   market-history-2016-01-20.csv.bz2
GET   market-history-2016-01-21.csv.bz2
GET   market-history-2016-01-22.csv.bz2
GET   market-history-2016-01-23.csv.bz2
GET   market-history-2016-01-24.csv.bz2
GET   market-history-2016-01-25.csv.bz2


In [13]:
# Read all bz2 files into a single dataframe
dfs = []
for file_path in OUT_DIR.glob("market-history-*.csv.bz2"):
    with bz2.open(file_path, "rt") as f:
        temp_df = pd.read_csv(f)
        # filter for The Forge
        temp_df = temp_df[temp_df["region_id"] == 10000002]
        temp_df = temp_df[[
            "average",
            "date",
            "highest",
            "lowest",
            "order_count",
            "volume",
            "type_id",
        ]]
        dfs.append(temp_df)

df: pd.DataFrame = pd.concat(dfs, ignore_index=True)

df

,average,date,highest,lowest,order_count,volume,type_id
0,63.33,2016-01-01,66.00,63.31,89,10775285,18
1,4500.02,2016-01-01,4500.02,4500.02,5,376,19
2,249.00,2016-01-01,249.99,230.58,111,1239579,20
3,499.01,2016-01-01,499.01,499.01,11,5632,21
4,3157.13,2016-01-01,3157.13,3157.13,15,19560,22
...,...,...,...,...,...,...,...
30449808,1689333.33,2026-01-15,1690000.00,1688000.00,3,3,91412
30449809,255820.00,2026-01-15,257200.00,250500.00,4,5,91413
30449810,215275.00,2026-01-15,215700.00,214000.00,3,4,91414
30449811,101571.43,2026-01-15,102000.00,101000.00,6,7,91415


In [14]:
# Downloaded from:
# https://data.everef.net/reference-data/
tar_path = Path("data/reference-data-latest.tar.xz")
def extract_labels():
    # unzip only once
    with tarfile.open(tar_path, "r:xz") as tar:
        types_file = tar.extractfile("types.json")
        regions_file = tar.extractfile("regions.json")

        types_data = yaml.safe_load(types_file)
        regions_data = yaml.safe_load(regions_file)
    types_df = pd.DataFrame([
        {"type_id": k, "type_name": i["name"]["en"]}
        for k, i in types_data.items()
    ])
    regions_df = pd.DataFrame([
        {"region_id": k, "region_name": i["name"]["en"]}
        for k, i in regions_data.items()
    ])
    # save pickles
    with open(OUT_DIR / "types_df.pkl", "wb") as f:
        pickle.dump(types_df, f)
    with open(OUT_DIR / "regions_df.pkl", "wb") as f:
        pickle.dump(regions_df, f)
    return types_df, regions_df

def get_labels():
    types_pkl = OUT_DIR / "types_df.pkl"
    regions_pkl = OUT_DIR / "regions_df.pkl"
    if types_pkl.exists() and regions_pkl.exists():
        with open(types_pkl, "rb") as f:
            types_df = pickle.load(f)
        with open(regions_pkl, "rb") as f:
            regions_df = pickle.load(f)
    else:
        types_df, regions_df = extract_labels()
    return types_df, regions_df

In [15]:
types_df, regions_df = get_labels()
regions_df

,region_id,region_name
0,10000001,Derelik
1,10000002,The Forge
2,10000003,Vale of the Silent
3,10000004,UUA-F4
4,10000005,Detorid
...,...,...
108,14000002,VR-02
109,14000003,VR-03
110,14000004,VR-04
111,14000005,VR-05


In [16]:
df["type_id"] = df["type_id"].astype(int)
types_df["type_id"] = types_df["type_id"].astype(int)

In [17]:
df = df.merge(types_df, on="type_id", how="left")
df

,average,date,highest,lowest,order_count,volume,type_id,type_name
0,63.33,2016-01-01,66.00,63.31,89,10775285,18,Plagioclase
1,4500.02,2016-01-01,4500.02,4500.02,5,376,19,Spodumain
2,249.00,2016-01-01,249.99,230.58,111,1239579,20,Kernite
3,499.01,2016-01-01,499.01,499.01,11,5632,21,Hedbergite
4,3157.13,2016-01-01,3157.13,3157.13,15,19560,22,Arkonor
...,...,...,...,...,...,...,...,...
30449808,1689333.33,2026-01-15,1690000.00,1688000.00,3,3,91412,"For Those Lost, But Never Forgotten"
30449809,255820.00,2026-01-15,257200.00,250500.00,4,5,91413,Canned Quantum Superposition Wormhole
30449810,215275.00,2026-01-15,215700.00,214000.00,3,4,91414,Scrap of paper
30449811,101571.43,2026-01-15,102000.00,101000.00,6,7,91415,The Capsuleer’s Confectionary Codex – Field Ed...


In [18]:
# pickle the forge dataframe
with open(OUT_DIR / "forge_market_history.pkl", "wb") as f:
    pickle.dump(df, f)